In [381]:
import pandas as pd
import itertools
import numpy as np
import openpyxl


In [382]:
fuel_dict = {
    1: (0.178, 0.094, 'fossil'),      # MGO
    2: (0.146, 0.073, 'fossil'),      # LNG
    3: (0.491, 0.261, 'green_fuels'), # GREEN METHANOL
    4: (0.324, 0.178, 'green_fuels'), # GREEN NH3
    5: (0.278, 0.147, 'bio'),         # B30
    6: (0.245, 0.130, 'bio'),         # B20
    7: (0.228, 0.121, 'bio'),         # B15
    8: (0.201, 0.106, 'bio'),         # B7
    9: (0.227, 0.108, 'green_fuels')  # EL40
}

t2_values = range(2,6)
w_values = range(1,11)


fuel_price_csv = 'fuel_price2tri_10s.csv'




In [383]:

# scenario_developments = [
#     {'fossil': [-0.026, -0.026, -0.026, -0.026], 'bio': [0.12, 0.12, 0.12,0.12], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]}, # high, high, high 
#     {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [0.12, 0.12, 0.12, 0.12], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]}, #LHH
#     {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [-0.051, -0.051, -0.051, -0.051], 'green_fuels': [-0.115, -0.115, -0.115, -0.115]}, 
#     {'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [0.038, 0.038, 0.038, 0.038], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]}
#     #{'fossil': [-0.113, -0.113, -0.113, -0.113], 'bio': [-0.051, -0.051, -0.051, -0.051], 'green_fuels': [-0.031, -0.031, -0.031, -0.031]}
#     # {'fossil': [-0.069, -0.069, -0.069, -0.069], 'bio': [0.038, 0.038, 0.038, 0.038], 'green_fuels': [-0.073, -0.073, -0.073, -0.073]}
# ]



In [384]:
# # Leser inn nødvendig MJ på å kjøre hver rute
mj_route = pd.read_csv('../route_generation/generated_datafiles_reduced/mj_route.csv')
#mj_route_el = pd.read_csv('../route_generation/generated_datafiles/mj_route_strøm.csv')

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
#distance_dict = distance_dict['mj_route']

# lager egne distanser for elektrisk

#specific_distance_dict = mj_route_el['mj_route'].to_dict()
#specific_distance_dict= specific_distance_dict['mj_route']
#electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(distance_dict)
mj_route

#print(electric_distance_dict)

{1: 221758.4, 2: 178953.6, 3: 192976.8, 4: 220346.8, 5: 210389.2, 6: 200084.8, 7: 265296.4, 8: 244674.0, 9: 240312.0, 10: 201093.2, 11: 205730.8, 12: 194600.4, 13: 193478.0, 14: 204507.6, 15: 280065.2, 16: 226675.6, 17: 240136.4, 18: 215881.2, 19: 241484.4, 20: 231120.4, 21: 233664.8, 22: 279163.6, 23: 212046.4, 24: 207487.2, 25: 227290.8, 26: 202117.2, 27: 182802.0, 28: 249453.2, 29: 261579.6, 30: 254460.8, 31: 225621.2, 32: 242612.0, 33: 229348.0, 34: 266538.8, 35: 253531.6, 36: 241464.4, 37: 266200.0, 38: 262597.2, 39: 243741.2, 40: 183554.4, 41: 206008.0, 42: 259612.4, 43: 257894.4, 44: 212159.2}


,mj_route
0,221758.4
1,178953.6
2,192976.8
3,220346.8
4,210389.2
5,200084.8
6,265296.4
7,244674.0
8,240312.0
9,201093.2


In [385]:
# def generate_fuel_prices(scenario_range, period_range, fuel_dict):
#     num_scenarios = len(scenario_range)
#     num_periods = len(period_range)
#     num_fuels = len(fuel_dict)
    
#     fuel_prices = np.zeros((num_fuels, num_periods, num_scenarios))
    
#     group_scaling_factors = {}
    
#     for _, (lower_bound, upper_bound, group) in fuel_dict.items():
#         group_scaling_factors[group] = np.ones((num_scenarios, num_periods))  # Initialize scaling factors to ones
    
#     # Apply scenario developments
#     for s_idx, scenario_dev in enumerate(scenario_developments):
#         print('s_idx ', s_idx, 'scenario_dev', scenario_dev)
#         for group, developments in scenario_dev.items():
#             if group in group_scaling_factors:
#                 for p_idx, period_dev in enumerate(developments):
#                     print('p_idx: ', p_idx, 'period_dev ', period_dev)
#                     if p_idx >= 1:  # Start applying incremental developments from the third period onwards
#                         group_scaling_factors[group][s_idx, p_idx:] += period_dev
    
#     for i, (fuel_key, (lower_bound, upper_bound, group)) in enumerate(fuel_dict.items()):
#         base_price = lower_bound + (upper_bound - lower_bound)
#         for s_idx, scenario in enumerate(scenario_range):
#             for p_idx, period in enumerate(period_range):
#                 print(group_scaling_factors[group][s_idx, p_idx])
#                 fuel_price = base_price * group_scaling_factors[group][s_idx, p_idx]
#                 fuel_prices[fuel_key - 1, p_idx, s_idx] = fuel_price  # Adjusted for zero-based indexing
    
#     return fuel_prices








In [386]:
def generate_fuel_prices(scenario_range, period_range, fuel_dict):
    num_scenarios = len(scenario_range)
    num_periods = len(period_range)
    num_fuels = len(fuel_dict)
    
    # Generate random scaling factors for each group, scenario, and time period
    scaling_factors = np.random.triangular(left=0, mode=0.5, right=1, size=(num_scenarios, num_periods))
    # Adjust the scaling factors based on correlation
    fuel_prices = np.zeros((num_fuels, num_periods, num_scenarios))
    
    # Create a dictionary to map group index to scaling factor
    group_scaling_factors = {}
    for group in set(group for _, (_, _, group) in fuel_dict.items()):
        group_scaling_factors[group] = np.random.triangular(left=0, mode=0.5, right=1)

    for i, (fuel_key, (lower_bound, upper_bound, group)) in enumerate(fuel_dict.items()):
        # Get the scaling factor for the fuel's group
        group_scaling_factor = group_scaling_factors[group]
        
        # Determine the group index based on the fuel's group
        for s_idx, scenario in enumerate(scenario_range):
            for p_idx, period in enumerate(period_range):
                # Obtain the scaling factor for the current scenario and time period
                scaling_factor = scaling_factors[s_idx, p_idx]
                # Calculate the fuel price based on the triangular distribution bounds and scaling factors
                fuel_price = lower_bound + (upper_bound - lower_bound) * scaling_factor * group_scaling_factor
                # Adjust fuel price based on previous period's price (if applicable)
                if p_idx > 0:
                    prev_fuel_price = fuel_prices[i, p_idx - 1, s_idx]
                    fuel_price = max(fuel_price, prev_fuel_price)
                # Store the calculated fuel price
                fuel_prices[i, p_idx, s_idx] = fuel_price
                
    return fuel_prices

In [387]:
fuel_prices = generate_fuel_prices(w_values, t2_values, fuel_dict)


# # Round fuel prices to two decimal places for display
# fuel_prices_rounded = np.round(fuel_prices, 2)
# print(fuel_prices_rounded)

In [388]:
# Generate fuel prices



# Convert fuel prices to a DataFrame
# fuel_prices_df = pd.DataFrame(columns=['F', 'R', 'T', 'O', 'Value'])
# for fuel_key, (lower_bound, upper_bound, _) in fuel_dict.items():
#     fuel_idx = list(fuel_dict.keys()).index(fuel_key)
#     for s_idx, scenario in enumerate(w_values):
#         for p_idx, period in enumerate(t2_values):
#             for route in r_values:
#                 distance = electric_distance_dict[route] if fuel_key == 6 else distance_dict[route]
#                 fuel_prices_df = pd.concat([fuel_prices_df, pd.DataFrame({
#                     'F': [fuel_key],
#                     'R': [route],
#                     'T': [period],
#                     'S': [scenario],
#                     'Value': [fuel_prices[fuel_idx, p_idx, s_idx] * distance]
#                 })], ignore_index=True)

# Convert fuel prices to a list of dictionaries
fuel_prices_data = []

for fuel_key, (lower_bound, upper_bound, _) in fuel_dict.items():
    fuel_idx = list(fuel_dict.keys()).index(fuel_key)
    for s_idx, scenario in enumerate(w_values):
        for p_idx, period in enumerate(t2_values):
            fuel_prices_data.append({
                'F': fuel_key,
                'T': period,
                'S': scenario,
                'Value': fuel_prices[fuel_idx, p_idx, s_idx] 
            })

# Create DataFrame from the list of dictionaries
fuel_prices_df = pd.DataFrame(fuel_prices_data)




print(fuel_prices_df)

fuel_prices_df.to_csv(fuel_price_csv, index=False)

     F  T   S     Value
0    1  2   1  0.174365
1    1  3   1  0.174365
2    1  4   1  0.174365
3    1  5   1  0.174365
4    1  2   2  0.154394
..  .. ..  ..       ...
355  9  5   9  0.217465
356  9  2  10  0.189830
357  9  3  10  0.198128
358  9  4  10  0.209946
359  9  5  10  0.212215

[360 rows x 4 columns]


In [389]:
result = fuel_prices_df[(fuel_prices_df['F'] == 1)]
print(result)

    F  T   S     Value
0   1  2   1  0.174365
1   1  3   1  0.174365
2   1  4   1  0.174365
3   1  5   1  0.174365
4   1  2   2  0.154394
5   1  3   2  0.154394
6   1  4   2  0.154394
7   1  5   2  0.159237
8   1  2   3  0.170155
9   1  3   3  0.170155
10  1  4   3  0.170155
11  1  5   3  0.170155
12  1  2   4  0.132052
13  1  3   4  0.156607
14  1  4   4  0.157110
15  1  5   4  0.157110
16  1  2   5  0.140963
17  1  3   5  0.165283
18  1  4   5  0.173341
19  1  5   5  0.173341
20  1  2   6  0.162068
21  1  3   6  0.162068
22  1  4   6  0.170602
23  1  5   6  0.170602
24  1  2   7  0.172162
25  1  3   7  0.172162
26  1  4   7  0.172162
27  1  5   7  0.172162
28  1  2   8  0.145083
29  1  3   8  0.164913
30  1  4   8  0.164913
31  1  5   8  0.164913
32  1  2   9  0.150157
33  1  3   9  0.150157
34  1  4   9  0.157757
35  1  5   9  0.170471
36  1  2  10  0.148650
37  1  3  10  0.155202
38  1  4  10  0.164534
39  1  5  10  0.166326
